In [4]:
import pandas as pd
import numpy as np
import json
# import matplotlib.pyplot as plt
# from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.endpoints import playercareerstats, teamgamelogs, playergamelogs
from nba_api.stats.static import players, teams

To Do:
[] 'hustle stats' game matching: https://github.com/swar/nba_api/blob/master/src/nba_api/stats/endpoints/hustlestatsboxscore.py

## Get Team Gamelogs

In [5]:
def get_team_gamelog_data(team_id, n_games, season):
    # https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/teamgamelogs.md
    team_gamelog_data = teamgamelogs.TeamGameLogs(team_id_nullable=team_id,
                                                  season_nullable=season,
                                                  last_n_games_nullable=n_games,
                                                 ).get_json()
    
    data_dict = json.loads(team_gamelog_data)
    df = pd.DataFrame(data_dict['resultSets'][0]['rowSet'], 
                      columns=data_dict['resultSets'][0]['headers'])
    
    df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
    df = df.set_index(['GAME_DATE', 'GAME_ID', 'TEAM_ABBREVIATION'])
    
    
    return df

In [6]:
teams_dict = teams.get_teams()
n_games = 82
seasons = ['2012-13',
           '2013-14',
           '2014-15',
           '2015-16',
           '2016-17',
           '2017-18',
           '2018-19',
           '2019-20',
           '2020-21',
           '2021-22',
           '2022-23',
           '2023-24']

df = pd.DataFrame()
for season in seasons:
    print(season)
    for i in range(0,len(teams_dict)):
        df = pd.concat([df, get_team_gamelog_data(i, n_games, season)], axis=0)

2012-13
2013-14
2014-15
2015-16
2016-17
2017-18
2018-19
2019-20
2020-21
2021-22
2022-23
2023-24


In [7]:
# SAVE...use sql in future...
df.to_csv('historical_stats_raw.csv')

## Processing

In [8]:
# factor encoding
df['Home'] = df['MATCHUP'].apply(lambda x: 0 if '@' in x else 1)
df['Win'] = df['WL'].map({'W': 1, 'L': 0})

# calculate per-48 stats, corrects for overtime games
cols = ['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 
        'FTA','OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 
        'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS']

new_cols = []
for col in cols:
    new_col_name = col + '_per48'
    df[new_col_name] = df[col]/df['MIN']*48 #np.round((df[col]/df['MIN'])*48, 2)
    new_cols.append(new_col_name)

df[new_cols] = np.round(df[new_cols], 2)

### Stats Against

In [9]:
cols = [      'Win',              'FGM_per48',        'FGA_per48',
              'FG3M_per48',       'FG3A_per48',
              'FTM_per48',        'FTA_per48',
              'OREB_per48',       'DREB_per48',
              'REB_per48',        'AST_per48',        'TOV_per48',
              'STL_per48',        'BLK_per48',        'BLKA_per48',
              'PF_per48',         'PFD_per48',        'PTS_per48',
              'FG_PCT',           'FG3_PCT']

for game_id in df.index.get_level_values('GAME_ID').unique():
    
    team_abvs = df.loc[:,game_id,:].index.get_level_values('TEAM_ABBREVIATION')
    if len(team_abvs) < 2:
        print('no opponent match')
    else:
        for col in cols:
            col_name = col + '_against'

            df.loc[(slice(None), game_id, team_abvs[0]), 
                            col_name] = df.loc[(slice(None), 
                                                     game_id, team_abvs[1])][col].iloc[0]
               
            df.loc[(slice(None), game_id, team_abvs[1]), 
                            col_name] = df.loc[(slice(None), 
                                                     game_id, team_abvs[0])][col].iloc[0]
               
df

SEASON_YEAR     TEAM_ID  \
GAME_DATE  GAME_ID    TEAM_ABBREVIATION                           
2013-04-17 0021201229 LAC                   2012-13  1610612746   
           0021201218 ATL                   2012-13  1610612737   
           0021201230 GSW                   2012-13  1610612744   
           0021201222 CHI                   2012-13  1610612741   
           0021201216 BOS                   2012-13  1610612738   
...                                             ...         ...   
2023-10-25 0022300071 MEM                   2023-24  1610612763   
2023-10-24 0022300062 GSW                   2023-24  1610612744   
                      PHX                   2023-24  1610612756   
           0022300061 LAL                   2023-24  1610612747   
                      DEN                   2023-24  1610612743   

                                                     TEAM_NAME      MATCHUP  \
GAME_DATE  GAME_ID    TEAM_ABBREVIATION                                       
2013-04-17 0021201229 LAC                 Los Angeles Clippers    LAC @ SAC   
           0021201218 ATL                        Atlanta Hawks    ATL @ NYK   
           0021201230 GSW                Golden State Warriors    GSW @ POR   
           0021201222 CHI                        Chicago Bulls  CHI vs. WAS   
           0021201216 BOS                       Boston Celtics    BOS @ TOR   
...                                                        ...          ...   
2023-10-25 0022300071 MEM                    Memphis Grizzlies  MEM vs. NOP   
2023-10-24 0022300062 GSW                Golden State Warriors  GSW vs. PHX   
                      PHX                         Phoenix Suns    PHX @ GSW   
           0022300061 LAL                   Los Angeles Lakers    LAL @ DEN   
                      DEN                       Denver Nuggets  DEN vs. LAL   

                                        WL   MIN FGM  FGA  FG_PCT FG3M  ...  \
GAME_DATE  GAME_ID    TEAM_ABBREVIATION                                 ...   
2013-04-17 0021201229 LAC                W  48.0  44   88   0.500   12  ...   
           0021201218 ATL                L  48.0  37   90   0.411    2  ...   
           0021201230 GSW                W  48.0  40   93   0.430    9  ...   
           0021201222 CHI                W  48.0  37   78   0.474    7  ...   
           0021201216 BOS                L  48.0  37   84   0.440    7  ...   
...                                     ..   ...  ..  ...     ...  ...  ...   
2023-10-25 0022300071 MEM                L  48.0  38   91   0.418   12  ...   
2023-10-24 0022300062 GSW                L  48.0  36  101   0.356   10  ...   
                      PHX                W  48.0  42   95   0.442   11  ...   
           0022300061 LAL                L  48.0  41   90   0.456   10  ...   
                      DEN                W  48.0  48   91   0.527   14  ...   

                                        AST_per48_against  TOV_per48_against  \
GAME_DATE  GAME_ID    TEAM_ABBREVIATION                                        
2013-04-17 0021201229 LAC                            23.0               11.0   
           0021201218 ATL                            21.0               12.0   
           0021201230 GSW                            18.0               14.0   
           0021201222 CHI                            17.0                9.0   
           0021201216 BOS                            26.0               11.0   
...                                                   ...                ...   
2023-10-25 0022300071 MEM                            22.0               21.0   
2023-10-24 0022300062 GSW                            23.0               19.0   
                      PHX                            19.0               11.0   
           0022300061 LAL                            29.0               12.0   
                      DEN                            23.0               12.0   

                                        STL_per48_against BLK_per48_aga

In [10]:
# SAVE for Feature Prep
df.to_csv('historical_stats.csv')